#1: Import thư viện

In [8]:
import math
from collections import defaultdict
from underthesea import word_tokenize  # Tokenizer tiếng Việt chuẩn

#2: Dữ liệu mẫu spam/ham

In [9]:
data = [
    ("Bạn đã trúng thưởng 100 triệu! Nhấp vào link nhận ngay!", "spam"),
    ("Giảm giá 50% cho đơn hàng hôm nay", "spam"),
    ("Nhận quà tặng miễn phí, click để nhận", "spam"),
    ("Mời bạn tham dự hội thảo về AI tuần tới", "ham"),
    ("Cuộc họp nhóm lúc 14h chiều nay", "ham"),
    ("Báo cáo tài chính đã được gửi qua email", "ham")
]

#3: Tiền xử lí text

In [10]:
def preprocess(text):
    return " ".join(text.strip().split())

def tokenize(text):
    return word_tokenize(preprocess(text.lower()), format="text").split()


# Test tokenize
sample_text = "Nhận quà tặng miễn phí, click để nhận"
print("Mẫu văn bản: ", sample_text)
print("Sau khi tiền xử lí:", tokenize(sample_text))

Mẫu văn bản:  Nhận quà tặng miễn phí, click để nhận
Sau khi tiền xử lí: ['nhận', 'quà', 'tặng', 'miễn_phí', ',', 'click', 'để', 'nhận']


#4: Bộ đếm Naive Bayes

In [11]:
class_counts = defaultdict(int)
word_counts = defaultdict(lambda: defaultdict(int))
total_words = defaultdict(int)

for text, label in data:
    class_counts[label] += 1
    words = tokenize(text)
    for word in words:
        word_counts[label][word] += 1
        total_words[label] += 1

print("Số email theo lớp:", dict(class_counts))
print("Tổng số từ mỗi lớp:", dict(total_words))
print("Một phần word_counts cho spam:", dict(list(word_counts['spam'].items())[:5]))

Số email theo lớp: {'spam': 3, 'ham': 3}
Tổng số từ mỗi lớp: {'spam': 28, 'ham': 23}
Một phần word_counts cho spam: {'bạn': 1, 'đã': 1, 'trúng': 1, 'thưởng': 1, '100': 1}


#5: Tính xác suất

In [13]:
def predict_proba(email, verbose=True):
    words = tokenize(email)
    scores = {}
    vocab = set()
    for label in class_counts:
        for word in word_counts[label]:
            vocab.add(word)
    V = len(vocab)

    for label in class_counts:
        scores[label] = math.log(class_counts[label] / sum(class_counts.values()))
        for word in words:
            count_w = word_counts[label].get(word, 0)
            scores[label] += math.log((count_w + 1) / (total_words[label] + V))

    # Chuyển log-probability -> xác suất chuẩn hóa 0-1
    max_log = max(scores.values())
    exp_scores = {label: math.exp(scores[label] - max_log) for label in scores}
    sum_exp = sum(exp_scores.values())
    probs = {label: exp_scores[label]/sum_exp for label in exp_scores}

    if verbose:
        for label, p in probs.items():
            print(f"Xác suất {label}: {p:.4f}")

    # Lớp dự đoán
    pred_label = max(probs, key=probs.get)
    return pred_label, probs

#6: Test

In [14]:
test_emails = [
    "Nhận quà miễn phí hôm nay",
]

for e in test_emails:
    print(f"\nEmail: '{e}'")
    pred_label, probs = predict_proba(e, verbose=True)
    print(f"Lớp dự đoán: {pred_label}")


Email: 'Nhận quà miễn phí hôm nay'
Xác suất spam: 0.9603
Xác suất ham: 0.0397
Lớp dự đoán: spam


#7: Debug chi tiết từng bước tính toán Naive Bayes

In [17]:
#7: Phân tích chi tiết xác suất từng bước cho email test
for email in test_emails:
    print("=" * 80)
    print(f"Email kiểm tra: '{email}'")
    words = tokenize(email)
    print(f"Tách từ: {words}\n")

    vocab = set()
    for label in class_counts:
        vocab |= set(word_counts[label].keys())
    V = len(vocab)

    print(f"Tổng số lớp: {len(class_counts)} -> {list(class_counts.keys())}")
    print(f"Kích thước từ vựng V = {V}\n")

    # Bảng chi tiết
    for label in class_counts:
        print(f"--- Lớp: {label.upper()} ---")
        prior = class_counts[label] / sum(class_counts.values())
        print(f"Prior P({label}) = {prior:.4f}")

        total_w = total_words[label]
        print(f"Tổng số từ trong lớp {label}: {total_w}\n")

        log_prob = math.log(prior)
        for w in words:
            count_w = word_counts[label].get(w, 0)
            cond_prob = (count_w + 1) / (total_w + V)
            log_prob += math.log(cond_prob)
            print(f"P({w}|{label}) = ({count_w} + 1) / ({total_w} + {V}) = {cond_prob:.6f}")

        print(f"→ Log-prob tổng cho lớp {label}: {log_prob:.6f}\n")

    # Dự đoán cuối cùng
    pred_label, probs = predict_proba(email, verbose=True)
    print(f" Kết luận: '{email}' được dự đoán là {pred_label.upper()}\n")


Email kiểm tra: 'Nhận quà miễn phí hôm nay'
Tách từ: ['nhận', 'quà', 'miễn_phí', 'hôm_nay']

Tổng số lớp: 2 -> ['spam', 'ham']
Kích thước từ vựng V = 46

--- Lớp: SPAM ---
Prior P(spam) = 0.5000
Tổng số từ trong lớp spam: 28

P(nhận|spam) = (3 + 1) / (28 + 46) = 0.054054
P(quà|spam) = (1 + 1) / (28 + 46) = 0.027027
P(miễn_phí|spam) = (1 + 1) / (28 + 46) = 0.027027
P(hôm_nay|spam) = (1 + 1) / (28 + 46) = 0.027027
→ Log-prob tổng cho lớp spam: -14.443672

--- Lớp: HAM ---
Prior P(ham) = 0.5000
Tổng số từ trong lớp ham: 23

P(nhận|ham) = (0 + 1) / (23 + 46) = 0.014493
P(quà|ham) = (0 + 1) / (23 + 46) = 0.014493
P(miễn_phí|ham) = (0 + 1) / (23 + 46) = 0.014493
P(hôm_nay|ham) = (0 + 1) / (23 + 46) = 0.014493
→ Log-prob tổng cho lớp ham: -17.629573

Xác suất spam: 0.9603
Xác suất ham: 0.0397
 Kết luận: 'Nhận quà miễn phí hôm nay' được dự đoán là **SPAM**

